In [1]:
import torch; torch.version.cuda
import torch; torch.__version__

'2.1.0+cu121'

In [ ]:
!pip install --upgrade --force-reinstall --no-cache-dir torch==2.1.0 triton --index-url https://download.pytorch.org/whl/cu121
!pip install --upgrade pip
!pip install --no-deps --no-build-isolation packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes

In [2]:
!pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-3jukznyu/unsloth_ab19cd172a8044ecb80b4b9265e1e0c1
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-3jukznyu/unsloth_ab19cd172a8044ecb80b4b9265e1e0c1
  Resolved https://github.com/unslothai/unsloth.git to commit bb81079ca1dba43fc2cdb79a81ce6edf23f87907
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 6.1 MB/s eta 0:00:0000:0100:01
  Created wheel for unsloth: filename=unsloth-2024.3-py3-none-any.whl size=90819 sha256=f33824c6d0cb9f7b513c4ed435bd81131d0c9a4e40ab20f960b7c43472efbfb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-xb0lhmjw/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
  Attempting uninstall: xformers
    F

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

/home/mtman/Documents/Repos/yakwith.ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/mtman/Documents/Repos/yakwith.ai/models/Mistral-7B-OpenOrca",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.65 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.24s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
model

In [5]:
import datasets

from datasets import load_dataset
def format_text(text):
    return {'text':str(text)}
dataset = load_dataset('csv', data_files='/home/mtman/Documents/Repos/yakwith.ai/data/1711005768.173661_2b5/completion_data.tsv', split='train')
dataset = dataset.map(format_text)

Generating train split: 41 examples [00:00, 12289.46 examples/s]
Map: 100%|██████████| 41/41 [00:00<00:00, 9099.24 examples/s]


In [6]:
len(dataset); dataset[0]['text']

'{\'<|im_start|>system\\nYou are a casual waiter in a restaurant. Your job is to collect food and drink orders from customers and answer their questions. You follow these RULES:  You do not make up facts about items on the menu.\\n            Where a menu item has choices and alternative ingredients you explore those with the customer before completing the order.\\n            You always read the order back to the customer at the end of the conversation and tell them the full price.\\n        . The MENU is as follows: {"food_items": [{"name": "BREKKY ROLL", "price": "$13", "descriptions": ["Bacon & Egg on a Milk Bun"], "other": ["Sauce: Tomato, BBQ, Chipotle Mayo or Aioli", "Add Cheese - $1, Smashed Avo - $2"]}, {"name": "EGGS POACHED OR FRIED", "price": "$13", "descriptions": ["Served on Sourdough or Ciabatta"], "other": ["Add Cheese - $1, Smashed Avo - $2"]}, {"name": "EGGS BENEDICT", "price": "$26", "descriptions": ["Two Poached Eggs", "Spinach", "Hollandaise Sauce", "Served on Sour

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 5,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 41 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 5 | Gradient Accumulation steps = 2
\        /    Total batch size = 10 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.462900
2,1.424000
3,1.227500
4,0.992600
5,0.866000
6,0.661900
7,0.501300
8,0.383700
9,0.261500
10,0.185100


In [10]:
FastLanguageModel.for_inference(model)

In [11]:
message = [{"role":"system",'content':'{""food_items"": [{""name"": ""EGGS POACHED OR FRIED"", ""price"": ""$13"", ""descriptions"": [""Served on Sourdough or Ciabatta""], ""other"": [""Add Cheese - $1, Smashed Avo - $2""]}, {""name"": ""RUSTIC TOAST"", ""price"": ""$8.50"", ""descriptions"": [""Sourdough or Ciabatta"", ""Fig & Lemon Jam"", ""Orange Marmalade"", ""Strawberry Jam"", ""Peanut Butter or Vegemite""]}, {""name"": ""MEXICAN eggs"", ""price"": ""$23"", ""descriptions"": [""Mixed Beans Cooked with Chili, Garlic, Onion and Herbs in a Rich Tomato Sauce"", ""Baked with Mozzarella"", ""Two Poached Eggs"", ""Ciabatta""]}, {""name"": ""HOT CHEESE"", ""price"": ""$9.50"", ""descriptions"": [""With Tomato & Oregano on Sourdough""]}], ""tea"": {""T-NOMICS ORGANIC LEAF TEA"": ""$5"", ""Varieties"": [""English Breakfast"", ""Earl Grey"", ""Peppermint"", ""Lemongrass & Ginger"", ""Golden Oolong Green""]}, ""immune_booster_shake"": {""price"": ""$14"", ""Ingredients"": [""Green apple"", ""Banana"", ""Spinach"", ""Mint"", ""Lime"", ""Ginger""]}, ""protein_shake"": {""price"": ""$14"", ""Ingredients"": [""Peanut butter"", ""Modern pea protein"", ""Almond milk"", ""Banana"", ""Honey & ice""], ""Additions"": [""Add shot of coffee $0.5""]}, ""extra_items"": {""Salmon, Gourmet Pork Sausages"": ""$6"", ""Bacon, Haloumi"": ""$5"", ""Avocado, Grilled Tomato"": ""$4"", ""Free-Range Egg"": ""$3"", ""Gluten-Free Bread"": ""$2""}}'},{"role":"user","content":'For the Hot Cheese, can I have haloumi instead of the regular cheese?'}]

In [18]:
message = [{"role":"system",'content':'{""food_items"": [{""name"": ""EGGS POACHED OR FRIED"", ""price"": ""$13"", ""descriptions"": [""Served on Sourdough or Ciabatta""], ""other"": [""Add Cheese - $1, Smashed Avo - $2""]}, {""name"": ""RUSTIC TOAST"", ""price"": ""$8.50"", ""descriptions"": [""Sourdough or Ciabatta"", ""Fig & Lemon Jam"", ""Orange Marmalade"", ""Strawberry Jam"", ""Peanut Butter or Vegemite""]}, {""name"": ""MEXICAN eggs"", ""price"": ""$23"", ""descriptions"": [""Mixed Beans Cooked with Chili, Garlic, Onion and Herbs in a Rich Tomato Sauce"", ""Baked with Mozzarella"", ""Two Poached Eggs"", ""Ciabatta""]}, {""name"": ""HOT CHEESE"", ""price"": ""$9.50"", ""descriptions"": [""With Tomato & Oregano on Sourdough""]}], ""tea"": {""T-NOMICS ORGANIC LEAF TEA"": ""$5"", ""Varieties"": [""English Breakfast"", ""Earl Grey"", ""Peppermint"", ""Lemongrass & Ginger"", ""Golden Oolong Green""]}, ""immune_booster_shake"": {""price"": ""$14"", ""Ingredients"": [""Green apple"", ""Banana"", ""Spinach"", ""Mint"", ""Lime"", ""Ginger""]}, ""protein_shake"": {""price"": ""$14"", ""Ingredients"": [""Peanut butter"", ""Modern pea protein"", ""Almond milk"", ""Banana"", ""Honey & ice""], ""Additions"": [""Add shot of coffee $0.5""]}, ""extra_items"": {""Salmon, Gourmet Pork Sausages"": ""$6"", ""Bacon, Haloumi"": ""$5"", ""Avocado, Grilled Tomato"": ""$4"", ""Free-Range Egg"": ""$3"", ""Gluten-Free Bread"": ""$2""}}'},{"role":"user","content":'Hi, can I just get the rustic toast thanks?'}]

In [13]:
message = [{"role":"system",'content':'{""food_items"": [{""name"": ""eggs poached or fried"", ""price"": ""$13"", ""descriptions"": [""Served on Sourdough or Ciabatta""], ""other"": [""Add Cheese - $1, Smashed Avo - $2""]}, {""name"": ""RUSTIC TOAST"", ""price"": ""$8.50"", ""descriptions"": [""Sourdough or Ciabatta"", ""Fig & Lemon Jam"", ""Orange Marmalade"", ""Strawberry Jam"", ""Peanut Butter or Vegemite""]}, {""name"": ""MEXICAN eggs"", ""price"": ""$23"", ""descriptions"": [""Mixed Beans Cooked with Chili, Garlic, Onion and Herbs in a Rich Tomato Sauce"", ""Baked with Mozzarella"", ""Two Poached Eggs"", ""Ciabatta""]}, {""name"": ""HOT CHEESE"", ""price"": ""$9.50"", ""descriptions"": [""With Tomato & Oregano on Sourdough""]}], ""tea"": {""T-NOMICS ORGANIC LEAF TEA"": ""$5"", ""Varieties"": [""English Breakfast"", ""Earl Grey"", ""Peppermint"", ""Lemongrass & Ginger"", ""Golden Oolong Green""]}, ""immune_booster_shake"": {""price"": ""$14"", ""Ingredients"": [""Green apple"", ""Banana"", ""Spinach"", ""Mint"", ""Lime"", ""Ginger""]}, ""protein_shake"": {""price"": ""$14"", ""Ingredients"": [""Peanut butter"", ""Modern pea protein"", ""Almond milk"", ""Banana"", ""Honey & ice""], ""Additions"": [""Add shot of coffee $0.5""]}, ""extra_items"": {""Salmon, Gourmet Pork Sausages"": ""$6"", ""Bacon, Haloumi"": ""$5"", ""Avocado, Grilled Tomato"": ""$4"", ""Free-Range Egg"": ""$3"", ""Gluten-Free Bread"": ""$2""}}'},{"role":"user","content":'Which teas are organic?'}]

In [14]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [19]:
inputs = tokenizer.apply_chat_template(message,tokenize=True, add_generation_prompt=True,return_tensors='pt')
input = inputs.to('cuda')
_ = model.generate(input_ids = input, streamer=text_streamer, max_new_tokens = 64, use_cache = True)

# tokenizer.batch_decode(outputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|> system
{""food_items"": [{""name"": ""EGGS POACHED OR FRIED"", ""price"": ""$13"", ""descriptions"": [""Served on Sourdough or Ciabatta""], ""other"": [""Add Cheese - $1, Smashed Avo - $2""]}, {""name"": ""RUSTIC TOAST"", ""price"": ""$8.50"", ""descriptions"": [""Sourdough or Ciabatta"", ""Fig & Lemon Jam"", ""Orange Marmalade"", ""Strawberry Jam"", ""Peanut Butter or Vegemite""]}, {""name"": ""MEXICAN eggs"", ""price"": ""$23"", ""descriptions"": [""Mixed Beans Cooked with Chili, Garlic, Onion and Herbs in a Rich Tomato Sauce"", ""Baked with Mozzarella"", ""Two Poached Eggs"", ""Ciabatta""]}, {""name"": ""HOT CHEESE"", ""price"": ""$9.50"", ""descriptions"": [""With Tomato & Oregano on Sourdough""]}], ""tea"": {""T-NOMICS ORGANIC LEAF TEA"": ""$5"", ""Varieties"": [""English Breakfast"", ""Earl Grey"", ""Peppermint"", ""Lemongrass & Ginger"", ""Golden Oolong Green""]}, ""immune_booster_shake"": {""price"": ""$14"", ""Ingredients"": [""Green apple"", ""Banana"", ""Spinach

In [20]:
model.save_pretrained_merged("/home/mtman/Documents/Repos/yakwith.ai/models/Finetuned-Mistral-7B-OpenOrca", tokenizer, save_method="merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 41.05 out of 61.94 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 112.11it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


: 

In [ ]:
model.generate